In [46]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


# 1. 数据预处理

In [41]:
testdf = pd.read_csv("YT_Videos_Comments.csv")
testdf.head()
testdf.columns

C:\Users\86139\AppData\Local\Temp\ipykernel_22924\610759299.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  testdf = pd.read_csv("YT_Videos_Comments.csv")


Index(['User', 'Video Title', 'Video Description', 'Video ID',
       'Comment (Displayed)', 'Comment (Actual)', 'Comment Author',
       'Comment Author Channel ID', 'Comment Time'],
      dtype='object')

## 1.1 选择Comment和Comment Author作为特征构建的主要内容
我的想法：对评论分词，然后对CommentAuthor计数，就这样

### 去缺失值

In [42]:
print(testdf.isnull().sum())

testdf = testdf.dropna(subset=['Comment (Actual)'])
testdf = testdf.dropna(subset=['Comment Author'])
testdf = testdf.loc[:,['User', 'Video Title', 'Comment (Actual)', 'Comment Author']]
testdf.head()

User                              0
Video Title                     149
Video Description            196767
Video ID                     429330
Comment (Displayed)          467375
Comment (Actual)             482881
Comment Author               482909
Comment Author Channel ID    482868
Comment Time                 482862
dtype: int64


,User,Video Title,Comment (Actual),Comment Author
0,Cleo Abram,"Robots made of spiders (yes, really)",zombie spider!! bomb the damn lab before it's ...,Bagus Hutomo
1,Cleo Abram,"Robots made of spiders (yes, really)","This is way less cool than it seems, spiders a...",CMZ neu
2,Cleo Abram,"Robots made of spiders (yes, really)",Spiders see this and this is why they made the...,Kiana Marrie
3,Cleo Abram,"Robots made of spiders (yes, really)",you looks pretty 😍,Noob
4,Cleo Abram,"Robots made of spiders (yes, really)",I can hear the hairs standing up on my wife’s ...,chancellor9000


### 构建计数变量

In [43]:
# 构建CommentAuthor计数向量
testdf['Comment Author Counts'] = testdf['Comment Author'].map(testdf['Comment Author'].value_counts())
print(testdf.shape)
print(testdf.isnull().sum())
testdf.head()


(379032, 5)
User                     0
Video Title              0
Comment (Actual)         0
Comment Author           0
Comment Author Counts    0
dtype: int64


,User,Video Title,Comment (Actual),Comment Author,Comment Author Counts
0,Cleo Abram,"Robots made of spiders (yes, really)",zombie spider!! bomb the damn lab before it's ...,Bagus Hutomo,1
1,Cleo Abram,"Robots made of spiders (yes, really)","This is way less cool than it seems, spiders a...",CMZ neu,2
2,Cleo Abram,"Robots made of spiders (yes, really)",Spiders see this and this is why they made the...,Kiana Marrie,1
3,Cleo Abram,"Robots made of spiders (yes, really)",you looks pretty 😍,Noob,6
4,Cleo Abram,"Robots made of spiders (yes, really)",I can hear the hairs standing up on my wife’s ...,chancellor9000,1


### 去除特殊符号

In [44]:
# def remove_punctuation(text):
#     pattern = re.compile(r'[^\w\s]')
#     return re.sub(pattern, '', text)

# testdf['Comment (Actual)'] = testdf['Comment (Actual)'].apply(remove_punctuation)

# testdf.head()

## 1.2 文本预处理，我照着gpt干的

In [47]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# 分词+去停用词函数，这边需要下载一个语料库
def text_segmentation(text):
    # Remove punctuation marks
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Remove emojis
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # Remove stopwords
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    
    # stemmer
    tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(tokens)

testdf["Comment Seg"] = testdf["Comment (Actual)"].apply(text_segmentation)
testdf.head()

,User,Video Title,Comment (Actual),Comment Author,Comment Author Counts,Comment Seg
0,Cleo Abram,"Robots made of spiders (yes, really)",zombie spider!! bomb the damn lab before it's ...,Bagus Hutomo,1,zombi spider bomb damn lab late
1,Cleo Abram,"Robots made of spiders (yes, really)","This is way less cool than it seems, spiders a...",CMZ neu,2,way less cool seem spider actual muscl extens ...
2,Cleo Abram,"Robots made of spiders (yes, really)",Spiders see this and this is why they made the...,Kiana Marrie,1,spider see made terrifi go poke panic button
3,Cleo Abram,"Robots made of spiders (yes, really)",you looks pretty 😍,Noob,6,look pretti
4,Cleo Abram,"Robots made of spiders (yes, really)",I can hear the hairs standing up on my wife’s ...,chancellor9000,1,hear hair stand wife arm mile away


# 1.3 保存

In [49]:
testdf.to_csv("preProcessdata.csv", index=False)
df = pd.read_csv("preProcessdata.csv")
df.head()

,User,Video Title,Comment (Actual),Comment Author,Comment Author Counts,Comment Seg
0,Cleo Abram,"Robots made of spiders (yes, really)",zombie spider!! bomb the damn lab before it's ...,Bagus Hutomo,1,zombi spider bomb damn lab late
1,Cleo Abram,"Robots made of spiders (yes, really)","This is way less cool than it seems, spiders a...",CMZ neu,2,way less cool seem spider actual muscl extens ...
2,Cleo Abram,"Robots made of spiders (yes, really)",Spiders see this and this is why they made the...,Kiana Marrie,1,spider see made terrifi go poke panic button
3,Cleo Abram,"Robots made of spiders (yes, really)",you looks pretty 😍,Noob,6,look pretti
4,Cleo Abram,"Robots made of spiders (yes, really)",I can hear the hairs standing up on my wife’s ...,chancellor9000,1,hear hair stand wife arm mile away
